# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,372.571450,0.594270,1234,373.165720,371.977180,746.331439,743.954361
6,473.659167,0.074482,1234,473.733649,473.584685,947.467298,947.169370
7,281.572109,0.519433,1234,282.091542,281.052677,564.183083,562.105353
8,179.117629,-0.456125,1234,178.661504,179.573754,357.323008,359.147508
9,142.680253,-0.699107,1234,141.981146,143.379360,283.962291,286.758720
10,454.887639,-0.268649,1234,454.618989,455.156288,909.237978,910.312576
11,414.713173,-0.736746,1234,413.976426,415.449919,827.952853,830.899837
12,299.142486,0.365942,1234,299.508428,298.776543,599.016856,597.553086
13,56.198797,-0.671077,1234,55.527720,56.869873,111.055440,113.739746
14,132.936588,0.647720,1234,133.584308,132.288868,267.168617,264.577735


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-373.165720,371.977180
6,-473.733649,473.584685
7,-282.091542,281.052677
8,-178.661504,179.573754
9,-141.981146,143.379360
10,-454.618989,455.156288
11,-413.976426,415.449919
12,-299.508428,298.776543
13,-55.527720,56.869873
14,-133.584308,132.288868


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-373.165720,371.977180
6,-473.733649,473.584685
7,-282.091542,281.052677
8,-178.661504,179.573754
9,-141.981146,143.379360
10,-454.618989,455.156288
11,-413.976426,415.449919
12,-299.508428,298.776543
13,-55.527720,56.869873
14,-133.584308,132.288868


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-373.16571962,  371.97718044],
       [-473.73364917,  473.58468511],
       [-282.09154174,  281.05267671],
       [-178.66150414,  179.57375394],
       [-141.98114561,  143.37935995],
       [-454.61898907,  455.15628796],
       [-413.97642637,  415.44991873],
       [-299.50842794,  298.7765431 ],
       [ -55.52771979,   56.86987324],
       [-133.58430832,  132.28886751]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,744.331439,-743.954361
6,945.467298,-947.169370
7,562.183083,-562.105353
8,355.323008,-359.147508
9,281.962291,-286.758720
10,907.237978,-910.312576
11,825.952853,-830.899837
12,597.016856,-597.553086
13,109.055440,-113.739746
14,265.168617,-264.577735


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.372571,0.797135
6,0.473659,0.537241
7,0.281572,0.759716
8,0.179118,0.271938
9,0.142680,0.150446
10,0.454888,0.365675
11,0.414713,0.131627
12,0.299142,0.682971
13,0.056199,0.164462
14,0.132937,0.823860


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,372.571450,0.594270
6,473.659167,0.074482
7,281.572109,0.519433
8,179.117629,-0.456125
9,142.680253,-0.699107
10,454.887639,-0.268649
11,414.713173,-0.736746
12,299.142486,0.365942
13,56.198797,-0.671077
14,132.936588,0.647720


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
